# World Health Organization Covid-19 Reporting

In [107]:
from bs4 import BeautifulSoup
import requests, wget, datetime, PyPDF2, os

In [12]:
site = "https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports"
req = requests.get(site)
soup = BeautifulSoup(req.content, 'html.parser')

In [13]:
sitrep = 'situation-reports' + '_' + str(datetime.date.today())
with open(sitrep, 'w') as f:
    f.writelines(req.text)

In [14]:
reports = [ 'https://www.who.int' + link for link in \
          [ l.get('href') for l in soup.find_all('a') ] \
          if link.startswith('/docs') ]

In [110]:
#verify all reports dl'd and not tmp'd or dup'd
#if reports all dl'd but for today, only get today
os.mkdir('reports')
def dl_todays_report():
    wget.download(reports[1],out='reports/')

def dl_all_reports():
    for r in reports:
        wget.download(r,out='reports/')

FileExistsError: [Errno 17] File exists: 'reports'

# Extract table data from pdf

SURVEILLANCE

Date

Area | Total Cases | New Cases | Total Deaths | New Deaths | Since Last Reported Case

In [18]:
#need programatic selection of filename
#database data
#process all files
#process newest/unprocessed files
rr_file = open('reports/20200331-sitrep-71-covid-19.pdf','rb')
rr = PyPDF2.PdfFileReader(rr_file)

In [19]:
page = ''
for p in range(rr.getNumPages()):
    page += rr.getPage(p).extractText()

In [222]:
#pull SURVEILLANCE BLOCK
#begins with 'SURVEILLANCE'
#ends with |^GRAND TOTAL| after \n 


# pull date
cases

and deaths

. Data as of

In [77]:
#pull date
def get_date():
    dotData=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('. Data') ][0]
    day = page.splitlines()[dotData+2]
    month = page.splitlines()[dotData+4]
    year = page.splitlines()[dotData+6]
    date = "%s-%s-%s" % (month,day,year)
    return date

# pull table data

In [20]:
#pull table data
report_begin=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('reported case') ][0] +2
report_end=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('Numbers') ][0] 
#surveillance_data = [ c.strip('\"') for c in \
#                    [ l for l in page.splitlines()[report_begin:report_end] if l != ' ' ] ]
surveillance_data = [ l for l in page.splitlines()[report_begin:report_end] if l != ' ' ]


In [ ]:
#if 'REGION' in line, line = region
#if 'Local transmission' in line, line[+2] = loc_trans 
#  and line[+3] = end

# extract by identification relativity

In [191]:
def get_data():
    result = {}
    for l in surveillance_data:
        region = ''
        area = ''
        if 'Region' in l:
            region = l
        else:
            region = region
            for c in l:
                if c.isalpha() or c.isspace():
                    area += c
            if area != '':
                block_begin = surveillance_data.index(area)
                total_confirmed = surveillance_data[block_begin+1]
                total_new = surveillance_data[block_begin+2]
                total_deaths = surveillance_data[block_begin+3]
                total_new_deaths = surveillance_data[block_begin+4]
                result[region] =  [ area,total_confirmed,total_new,total_deaths,total_new_deaths]
    return result

In [32]:
for l in surveillance_data:
    region = ''
    area = ''
    trans = ''
    if 'Region' in l:
        region = l
    else:
        if region:
            if 'Imported' in l or 'Local' in l:
                trans = l
            else:
                for c in l:
                    if c.isalpha() or c.isspace():
                        area += c           
            if area:
                block_begin = surveillance_data.index(area)
                total_confirmed = surveillance_data[block_begin+1]
                total_new = surveillance_data[block_begin+2]
                total_deaths = surveillance_data[block_begin+3]
                total_new_deaths = surveillance_data[block_begin+4]
                print(region,[ area,total_confirmed,total_new,total_deaths,total_new_deaths])

# extract by indexing?

In [105]:
def region_index():
    region_index = {}
    for ri in surveillance_data:
        if 'Region' in ri:
            region_index[surveillance_data.index(ri)] = ri
            #region_index.append(surveillance_data.index(ri))
    ri = [ l for l in sorted(region_index.items()) ]
    return ri

In [106]:
def area_block(ri):
    idx_map = {}
    for i in range(0,len(ri)-1):
        for idx in range(ri[i][0],ri[i+1][0]):
            if idx == ri[-1][0]-1:
                idx_map[ri[-1][1]] = ri[-1][0]
            #print(ri[i][1], 'range:',idx)
            idx_map[ri[i][1]] = [ i for i in range(ri[i][0],ri[i+1][0]) ] ,
    return idx_map

In [100]:
area_index = area_block(region_index())

In [104]:
[ a for a in area_index.keys() ][0]

'Western Pacific Region'